# Lab 1
This is the second jupyter notebook for lab 1. First we will import some libraries and functions. 

In [ ]:
import numpy as np
from netCDF4 import MFDataset 
from netCDF4 import Dataset 
import matplotlib
matplotlib.use('Agg')
import matplotlib.pylab as plt
from datetime import datetime
from netCDF4 import num2date
from netCDF4 import date2index
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import metpy.calc as mpcalc
from metpy.units import units
import xarray as xr
import cartopy.util as cutil
%matplotlib inline

Our next setep is to read in the netCDF files. 

In [ ]:
ds = xr.open_dataset('https://psl.noaa.gov/psd/thredds/dodsC/Datasets/ncep.reanalysis.derived/surface/air.mon.mean.nc')
ds2 = xr.open_dataset('https://psl.noaa.gov/psd/thredds/dodsC/Datasets/ncep.reanalysis.derived/surface/slp.mon.mean.nc')
ds3 = xr.open_dataset('https://psl.noaa.gov/psd/thredds/dodsC/Datasets/ncep.reanalysis.derived/surface/uwnd.mon.mean.nc')
ds4 = xr.open_dataset('https://psl.noaa.gov/psd/thredds/dodsC/Datasets/ncep.reanalysis.derived/surface/vwnd.mon.mean.nc')


Indicate the desired year and month!

In [ ]:
year = 1967
month = 1
monthlabel = 'January'

Extract lat, lon, temperature, SLP, uwnd and vwnd! We also calculate wind speed from u and v components

In [ ]:
# Grab lat/lon values
lats = ds.lat.data
lons = ds.lon.data
#print(lons)

# Grab air temperature, geopotential heights and wind components for desired pressure level (set to 500 hPa above)
temp = ds.air.sel(time=np.datetime64(datetime(year,month,1)))

slp = ds2.slp.sel(time=np.datetime64(datetime(year,month,1)))

uwnd = ds3.uwnd.sel(time=np.datetime64(datetime(year,month,1))).values[:] * units('m/s')

vwnd = ds4.vwnd.sel(time=np.datetime64(datetime(year,month,1))).values[:] * units('m/s')

# Use MetPy to calculate the wind speed for colorfill plot, change units to knots from m/s
# not plotted in the current application
# sped = mpcalc.get_wind_speed(uwnd, vwnd).to('kt')

# Create a clean datetime object for plot label
vtime = datetime(year,month,1)


Print the numpy shapes of the temperature, latitude and longtitude. This can be important when debugging.

In [ ]:
print (np.shape(temp))
print (np.shape(slp))
print (np.shape(lats))
print (np.shape(lons))

Select the countour settings for the temperature (in degrees C) and Z (in m). The format is (minimum, maximum, interval) 

In [ ]:
clevs_temp = np.arange(-50,50,5)
clevs_slp = np.arange(940,1080,4)

Make the global map!

In [ ]:
# Set up the projection that will be used for plotting
mapcrs = ccrs.Robinson(central_longitude=180)

# Set up the projection of the data; if lat/lon then PlateCarree is what you want
datacrs = ccrs.PlateCarree()

# Start the figure and create plot axes with proper projection
fig = plt.figure(1, figsize=(14, 12))
ax = plt.subplot(111, projection=mapcrs)

#not needed here, but used for sub-setting domain to plot
#ax.set_extent([-130, -72, 20, 55], ccrs.PlateCarree())

# Add geopolitical boundaries for map reference
ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax.add_feature(cfeature.STATES.with_scale('50m'))
gl = ax.gridlines(color='gray',linewidth=2, alpha = 0.5, linestyle='--')

# labels gridlines for PlateCarree ccrs option (doesn't work with Robinson, etc.)
#gl.xlabels_top = False
#gl.ylabels_right = False

#color-filled contour time!
#plotting temperature and including color bar
cf = ax.contourf(lons, lats, temp, clevs_temp, cmap=plt.cm.RdYlBu_r, transform=datacrs)
plt.colorbar(cf, orientation='horizontal', pad=0.05, aspect=50)

# Plot SLP
cs = ax.contour(lons, lats, slp, clevs_slp, colors='black', transform=datacrs)
plt.clabel(cs, fmt='%d')

# Plot wind barbs in knots, regrid to reduce number of barbs
ax.barbs(lons[::4], lats[::4], uwnd.to('kt')[::4,::4].m, vwnd.to('kt')[::4,::4].m, pivot='middle',
         color='black', transform=datacrs, length=5, linewidth=0.4)

# Make some nice titles for the plot (one right, one left)
plt.title('Surface Analysis', loc='left')
plt.title('Valid Time: {}'.format(vtime), loc='right')

# Reduce excessive white space around image and show
plt.savefig('SurfaceAnalysis' + str(monthlabel) + str(year), dpi=150, bbox_inches='tight')
plt.show()